# SEIRHVD model example
## Work in progress (equations not ready)
\begin{align}
\dot{S} & = S_f - \alpha\beta\frac{SI}{N+k_I I+k_R R} + r_{R\_S} R\\
\dot{E} & = E_f + \alpha\beta\frac{SI}{N+k_I I+k_R R} - E\frac{1}{t_{E\_I}} \\
\dot{I} & = I_f + E\frac{1}{t_{E\_I}} - I\frac{1}{t_{I\_R}} \\
\dot{R} & = R_f + I\frac{1}{t_{I\_R}} - r_{I\_R} R\\
\end{align}
Where:  
* $S:$ Susceptible
* $E:$ Exposed
* $I:$ Infectious
* $R:$ Removed
* $\alpha:$ Mobilty
* $\beta:$ Infection rate
* $N:$ Total population
* $t_{E\_I}:$ # Transition time between exposed and infectious
* $t_{I\_R}:$ # Transition time between infectious and recovered
* $r_{R\_S}:$ Immunity loss rate ($\frac{1}{t_{R\_S}}$)  
* $S_f,E_f,I_f,R_f:$ External flux
* $k_I:$ Infected saturation   
* $k_R:$ Immunity shield    


In [2]:
# Util libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
# Adding lib paths
import sys
sys.path.insert(1, '../src2/models/')
sys.path.insert(1, '../src2/utils/')
sys.path.insert(1, '../src2/data/')

# cv19 libraries
from SEIRHVD import SEIRHVD 
from SEIR import SEIR 
import cv19functions
from cv19data import ImportData

#sys.path.insert(1, '../src2/data/')
#sys.path.insert(1, '../src2/utils/')
#import cv19functions
#import cv19data

In [3]:
# For pop-up plots execute this code (optional)
import platform
OS = platform.system()

if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')

Linux


# Build a basic SEIR with SEIRHVD lib

import importlib
importlib.reload(cv19functions)
import SEIRHVD
importlib.reload(SEIRHVD)
from SEIRHVD import SEIRHVD


In [4]:
# Input configuration file
config = 'cfg/SEIRHVD_SEIR.toml'
# Build simulation object
model = SEIRHVD(config = config)

In [5]:
%%capture
# Simulate (solve ODE)
model.integrate()

In [6]:
# Show results
#model.results

In [8]:
# Plot matplotlib
plt.plot(model.t,model.S,label='S')
plt.plot(model.t,model.E,label='E')
plt.plot(model.t,model.I,label='I')
plt.plot(model.t,model.R,label='R')
plt.plot(model.t,model.H,label='H')
plt.plot(model.t,model.Iv,label='Iv')
plt.plot(model.t,model.D,label='D')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('SEIR from SEIRHVD lib')
plt.show()

In [128]:
%%capture
# Plot with pandas
seirconfig = 'cfg/SEIR.toml'
seirmodel = SEIR(config = seirconfig)# Input configuration file
seirmodel.integrate()

In [129]:
# Plot matplotlib
plt.plot(model.t,model.I,label='I_seirhvd')
plt.plot(seirmodel.t,seirmodel.I,label='I_seir')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('SEIR and SEIRHVD Comparison')
plt.show()

# Vaccines

### Simple SEIR with vaccines

In [9]:
# Vaccine campaign
Psi = cv19functions.events(values=[0,20000,0],days=[[0,40],[40,60],[60,500]])

In [10]:
%%capture
# Input configuration file
config = 'cfg/SEIRHVD_SEIR.toml'
# Build simulation object
model_vac = SEIRHVD(config = config,Psi=Psi,v = 1.0)
# Simulate (solve ODE)
model_vac.integrate()

In [11]:
# Plot matplotlib

plt.plot(model_vac.t,model_vac.I,label='I with vaccines')
plt.plot(model.t,model.I,label='I no vaccines')
plt.plot(model_vac.t,model_vac.Iv,label='Vaccinated infected')
plt.plot(np.arange(model_vac.tsim),[Psi(i) for i in np.arange(model_vac.tsim)])
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('SEIRV')
plt.show()

## Vaccines effectivity

In [12]:
v = list(np.arange(0,1.1,0.1))
model_vac2 = [SEIRHVD(config = config,Psi=Psi,v = i) for i in v]
for i in model_vac2:
    i.integrate()

In [13]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(v)))

In [14]:
i = 0
for model_vac in model_vac2:
    #plt.plot(model_vac.t,model_vac.I,label='I with vaccines')
    "{:#.2g}".format(i)
    plt.plot(model_vac.t,model_vac.I,label='I eff:'+"{:#.1g}".format(model_vac.v(0)),color=colors[i])
    #plt.plot(model_vac.t,model_vac.Iv,label='Iv eff:'+"{:#.1g}".format(model_vac.v(0)),color=colors[i],linestyle='dashed')
    i+=1
plt.plot(np.arange(model_vac.tsim),[Psi(i) for i in np.arange(model_vac.tsim)])
plt.legend(loc=0)

plt.xlim(0,300)
plt.title('Vaccines effectivity')
plt.show() 

# Hospitalization

In [15]:
%%capture
# Input configuration file
config = 'cfg/SEIRHVD.toml'
# Build simulation object
model1 = SEIRHVD(config = config, H_cap=4000)
model2 = SEIRHVD(config = config, H_cap=3500,Psi=Psi)
# Simulate (solve ODE)
model1.integrate()
model2.integrate()

In [17]:
# Plot matplotlib
#plt.plot(model.t,model.S,label='S')
#plt.plot(model.t,model.E,label='E')
plt.plot(model1.t,model1.I,label='I')
plt.plot(model1.t,model1.Im,label='Im')
plt.plot(model1.t,model1.Icr,label='Icr')
#plt.plot(model.t,model.R,label='R')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('SEIR from SEIRHVD lib')
plt.show()

In [19]:
# Plot matplotlib
plt.plot(model1.t,model1.H,color='tab:red',label='H 1')
plt.plot(model1.t,model1.D,color='tab:red',linestyle='dotted',label='D 1')
plt.plot(model1.t,[model1.H_cap]*len(model.t),color='tab:red',linestyle='dashed',label='H_cap 1')

plt.plot(model2.t,model2.H,color='tab:blue',label='H 2')
plt.plot(model2.t,model2.D,color='tab:blue',linestyle='dotted',label='D 2')
plt.plot(model2.t,[model2.H_cap]*len(model.t),color='tab:blue',linestyle='dashed',label='H_cap 2')
#plt.xlim(0,300)
plt.plot(np.arange(model_vac.tsim),[Psi(i) for i in np.arange(model_vac.tsim)])
plt.legend(loc=0)
plt.title('ICU capacity and deaths')
plt.show()

# Vaccines amount, effectivity  and deaths

In [112]:
eff = list(np.arange(0,1.1,0.1))
amount = np.linspace(0,30000,31)

In [114]:
# Vaccine campaign
Psi = [cv19functions.events(values=[0,i,0],days=[[0,40],[40,60],[60,500]]) for i in amount]

In [119]:
%%capture
# Input configuration file
config = 'cfg/SEIRHVD_SEIR.toml'
# Build simulation object
models = []
for i in eff:
    aux = []
    for j in Psi:
        a = SEIRHVD(config = config,Psi=j,v = i)
        a.integrate()
        aux.append(a)
    models.append(aux)
        # Simulate (solve ODE)
#model_vac.integrate()

In [116]:
a = SEIRHVD(config=config).integrate()

In [52]:
# Plot matplotlib

plt.plot(model_vac.t,model_vac.I,label='I with vaccines')
plt.plot(model.t,model.I,label='I no vaccines')
plt.plot(model_vac.t,model_vac.Iv,label='Vaccinated infected')
plt.plot(np.arange(model_vac.tsim),[Psi(i) for i in np.arange(model_vac.tsim)])
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('SEIRV')
plt.show()

In [60]:

model_vac2 = [SEIRHVD(config = config,Psi=Psi,v = i) for i in v]
for i in model_vac2:
    i.integrate()

In [61]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(v)))

In [62]:
i = 0
for model_vac in model_vac2:
    #plt.plot(model_vac.t,model_vac.I,label='I with vaccines')
    "{:#.2g}".format(i)
    plt.plot(model_vac.t,model_vac.I,label='I eff:'+"{:#.1g}".format(model_vac.v(0)),color=colors[i])
    #plt.plot(model_vac.t,model_vac.Iv,label='Iv eff:'+"{:#.1g}".format(model_vac.v(0)),color=colors[i],linestyle='dashed')
    i+=1
plt.plot(np.arange(model_vac.tsim),[Psi(i) for i in np.arange(model_vac.tsim)])
plt.legend(loc=0)

plt.xlim(0,300)
plt.title('Vaccines effectivity')
plt.show() 

# Hospitalization CFR variable

In [32]:
pH_R = cv19functions.events(values=[0.7,0.6],days=[[0,30],[30,500]])
pH_D = cv19functions.events(values=[0.3,0.4],days=[[0,30],[30,500]])

In [42]:
import scipy

In [ ]:
scipy.

In [ ]:
pH_D = sc

In [38]:
%%capture
# Input configuration file
config = 'cfg/SEIRHVD.toml'
# Build simulation object

model1 = SEIRHVD(config = config, H_cap=4000, pH_R = pH_R,pH_D = pH_D)
# Simulate (solve ODE)
model1.integrate()

In [39]:
# Plot matplotlib
#plt.plot(model.t,model.S,label='S')
#plt.plot(model.t,model.E,label='E')
plt.plot(model1.t,model1.I,label='I')
plt.plot(model1.t,model1.Im,label='Im')
plt.plot(model1.t,model1.Icr,label='Icr')
#plt.plot(model.t,model.R,label='R')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('SEIR from SEIRHVD lib')
plt.show()

In [41]:
# Plot matplotlib
#plt.plot(model1.t,model1.H,color='tab:red',label='H 1')
plt.plot(model1.t,model1.D_d,color='tab:red',linestyle='dotted',label='D 1')
#plt.plot(model1.t,[model1.H_cap]*len(model.t),color='tab:red',linestyle='dashed',label='H_cap 1')
plt.plot(model1.t,[pH_D(i) for i in model.t],color='tab:red',linestyle='dashed',label='pH_D')

#plt.plot(np.arange(model_vac.tsim),[Psi(i) for i in np.arange(model_vac.tsim)])
plt.legend(loc=0)
plt.title('ICU capacity and deaths')
plt.show()